In [818]:
import pandas as pd
import numpy as np
from vincent.colors import brews
from heapq import nlargest
pd.options.mode.chained_assignment = None
from scipy.stats import zscore

In [977]:
COMPANY = 'DSM'

CRITERIA = 'Criteria - sorted by importance'
IMP_MEAN = 'importance mean on a 10 pt scale'
EMPTY = 'empty'
MAX_VALUE = 'max_value'
COLOR = 'color'
COLOR_COL = 'Color of Columns'
BORDER_COL = 'Border of Column'
T_SCORE = 't_score'

In [820]:
def round_of_rating(number):
    """Round a number to the closest half integer."""
    return round(number * 2) / 2

In [976]:
print(mind_scan.columns)

Index(['Criteria - sorted by importance', 'importance mean on a 10 pt scale',
       'CP Kelco', 'DuPont Danisco', 'DSM', 'Fufeng', 'Jungbunzlauer (JBL)',
       'ADM', 'Cargill'],
      dtype='object')


In [978]:
# Initial grab of the data
mind_scan = pd.read_excel("Testing.xlsx", header=0, index_col=0)
mind_scan_no_base = mind_scan.loc[mind_scan.index.dropna()]

mind_scan_no_base[IMP_MEAN] = mind_scan_no_base[IMP_MEAN].round(1).copy()

values_to_add = []
for i in range(int(mind_scan_no_base[IMP_MEAN].min()*10), int(mind_scan_no_base[IMP_MEAN].max()*10)):
    if i/10 not in mind_scan_no_base[IMP_MEAN].values:
        values_to_add.append(i/10)


data_cols = list(mind_scan_no_base.columns.values)
data_cols.remove(COMPANY)
data_cols.insert(2, COMPANY)
data_cols[0], data_cols[1] = data_cols[1], data_cols[0]

In [979]:
values_df = pd.DataFrame(values_to_add, columns=[IMP_MEAN])
values_df[EMPTY] = np.empty((values_df.shape[0],1))
values_df[EMPTY] = np.nan
values_df = values_df.set_index(EMPTY)
mind_scan_no_base_all_x = mind_scan_no_base.append(values_df)
mind_scan_no_base_all_x = mind_scan_no_base_all_x.iloc[::-1]

mind_scan_no_base_all_x = mind_scan_no_base_all_x.sort_values(IMP_MEAN)
mind_scan_no_base_all_x = mind_scan_no_base_all_x[data_cols]

mind_values = mind_scan_no_base_all_x.drop([CRITERIA, IMP_MEAN], axis=1)

for x in range(mind_scan_no_base_all_x.shape[0]):
    if pd.isnull(mind_scan_no_base_all_x.index[x]):
        mind_scan_no_base_all_x[IMP_MEAN][x] = np.nan

mind_scan_no_base_all_x[MAX_VALUE] = (mind_values.dropna().values.max() + 0.2)

In [980]:
# Get number of data points above and to the right of the lines
y_list = [item for sublist in mind_values.dropna().values for item in sublist]
x_list = []
for x in mind_scan_no_base.index:
    for i in mind_values.columns:
        x_list.append(mind_scan_no_base[i].loc[x])

above_axis_x_num = int(mind_values.dropna().shape[0] * mind_values.dropna().shape[1] * 0.25)
right_axis_y_num = int(mind_values.dropna().shape[0] / 3)

top_x = nlargest(above_axis_x_num, x_list)[-1]
top_y = mind_scan_no_base_all_x[IMP_MEAN].dropna().iloc[-right_axis_y_num]
top_y_for_coloring = mind_scan_no_base_all_x[IMP_MEAN].dropna().iloc[-(right_axis_y_num + 1)]

In [981]:
x_limit = mind_scan_no_base_all_x.index.get_loc(mind_scan_no_base[mind_scan_no_base[IMP_MEAN] == (top_y)].index[-1])
x_limit_for_coloring = mind_scan_no_base_all_x.index.get_loc(
    mind_scan_no_base[mind_scan_no_base[IMP_MEAN] == (top_y_for_coloring)].index[-1])

In [982]:
column_colors = pd.DataFrame("#FFFFFF", columns=[COLOR_COL, BORDER_COL], index=mind_scan_no_base_all_x[IMP_MEAN])

decide_columns = mind_values.reset_index(drop=True).dropna().loc[x_limit_for_coloring:].T

for col in decide_columns.columns:
    decide_columns[T_SCORE] = zscore(decide_columns[col], ddof=1)
    
    # Open
    if decide_columns[col].max() <= top_x:
        column_colors[COLOR_COL].iloc[col] = '#FFF633'
        column_colors[BORDER_COL].iloc[col] = '#E3DB21'
        print('Open')
                
    #Owned
    elif decide_columns[T_SCORE].loc[COMPANY] > (decide_columns[T_SCORE].drop(COMPANY).max() + (
        abs(decide_columns[T_SCORE].drop(COMPANY).max()) + abs(decide_columns[
            T_SCORE].drop(COMPANY).min()))/decide_columns.drop(COMPANY).shape[0]):
        column_colors[COLOR_COL].iloc[col] = '#73D56B'
        column_colors[BORDER_COL].iloc[col] = '#57A051'
        print('Owned')

    # Battleground if the Company is close to max
    elif decide_columns[T_SCORE].loc[COMPANY] > 0:
        column_colors[COLOR_COL].iloc[col] = '#B2B1D5'
        column_colors[BORDER_COL].iloc[col] = '#9190AE'
        print('Battle')


list_of_column_colors = []
for a in range(column_colors.shape[0]):
    list_of_column_colors.append({'fill' : {'color' : column_colors[COLOR_COL].iloc[a]}, 
                                  'border': {'color': column_colors[BORDER_COL].iloc[a]}})

Open
Battle
Battle


In [985]:
series_to_delete = []
series_to_delete.append(0)
series_to_delete.append(1)
for i in range(mind_values.shape[1]):
    series_to_delete.append(mind_values.shape[1] +2 + i)

In [986]:
# Makes the new workbook and outputs the data into that one

excel_file = 'testing-output.xlsx'
sheet_name = 'Sheet1'

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
mind_scan_no_base_all_x.to_excel(writer, sheet_name=sheet_name)

# Access the XlsxWriter workbook and worksheet objects from the dataframe.
workbook = writer.book
worksheet = writer.sheets[sheet_name]

In [987]:
# Create a chart object.
scatter_chart = workbook.add_chart({'type': 'line'})
column_chart = workbook.add_chart({'type': 'column', 'subtype': 'stacked'})
chartsheet = workbook.add_chartsheet()

# Configure the series of the chart from the dataframe data.
max_row = mind_scan_no_base_all_x.shape[0]
max_column = mind_scan_no_base_all_x.shape[1]

for i in range(mind_values.shape[1]):
    col = i + 3
    
    scatter_chart.add_series({
        'name': ['Sheet1', 0, col],
        'categories' : ['Sheet1', 1, 0, max_row, 0],
        'values' : ['Sheet1', 1, col, max_row, col],
        'line': {'none': True},
        'marker' : {'type': 'automatic',
                   'size': 5,
                   'fill': {'color': brews['Set3'][i]}},
    })

for i in range(mind_values.shape[1]):
    col = i + 3
    scatter_chart.add_series({
        'categories' : ['Sheet1', 1, 0, max_row, 0],
        'values' : ['Sheet1', 1, col, max_row, col],
        'line': {'none': True},
        'x2_axis' : True,
        'y2_axis' : True,
    })


In [988]:
column_chart.add_series({
    'categories' : ['Sheet1', 1, 0, max_row, 0],
    'values' : ['Sheet1', 1, max_column, max_row, max_column],
    'gap':0,
    'points': list_of_column_colors,
})

column_chart.add_series({
    'categories' : ['Sheet1', 1, 0, max_row, 0],
    'values' : ['Sheet1', 1, 1, max_row, 1],
    'fill':   {'none': True},
    'data_labels': {'value': True,
                    'legend_key': False,
                    'position': 'inside_base',
                    'num_font':  {'size': 12},
                   },
})

scatter_chart.combine(column_chart)

# Configure the chart axes.
scatter_chart.set_x_axis({
    'num_font':  {'size': 12},
    'major_gridlines': {'visible': False},
    'major_tick_mark' : 'none',
    'minor_tick_mark' : 'none',
})

scatter_chart.set_x2_axis({
    'crossing' : x_limit,
    'visible' : True,
    'major_tick_mark' : 'none',
    'minor_tick_mark' : 'none',

})

scatter_chart.set_y_axis({
    'major_gridlines': {'visible': False},
    'minor_unit': 2, 
    'major_unit': 0.5,
    'max': (mind_values.dropna().values.max() + 0.2)+.01,
    'min': round_of_rating(mind_values.dropna().values.min() - 0.2),
    'num_format': '#,##0.00',
    #'num_font': {'rotation': -45}
})

scatter_chart.set_y2_axis({
    'crossing' : top_x,
    'major_tick_mark' : 'none',
    'minor_tick_mark' : 'none',
    'label_position': 'none',
    'max': (mind_values.dropna().values.max() + 0.2),
    'min': round_of_rating(mind_values.dropna().values.min() - 0.2),
})


# Position the legend at the top of the chart.
scatter_chart.set_legend({
    'position': 'top',
    'delete_series': series_to_delete,
})

scatter_chart.set_plotarea({
    'border': {'color': 'black', 'width': 1},
})

scatter_chart.set_size({'width': 1000, 'height': 800})

scatter_chart.set_title({
    'name': '2017 All Respondents',
    'name_font': {'size': 14,
                  'bold': True,
                  'color':'red',
                  'name':'Pinup'
                 },
})



# Insert the chart into the worksheet.
#worksheet.insert_chart('K2', scatter_chart)

chartsheet.set_chart(scatter_chart)

# Close the Pandas Excel writer and output the Excel file.
writer.save()